In [8]:
%reload_ext autoreload
%autoreload 2

# 1. 載入必要模組
import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm

# 載入自訂模組
from lib.backtest.strategy_one import backtest_strategy
from lib.performance_analysis import calculate_strategy_one_performance

print("所有模組載入成功。")

# 2. 載入資料
try:
    df_original = pd.read_csv("../data/TPE-sample1.csv", encoding="utf-8")
    print(f"資料載入成功，共 {len(df_original)} 筆。")
except FileNotFoundError:
    print("錯誤：找不到資料檔案 '../data/TPE-sample1.csv'。請確認路徑是否正確。")

# 3. 設定敏感度分析參數
iterations = 100
param_ranges = {
    'ma_period': (3, 15),
    'bb_period': (16, 40),
    'bb_std': (1.0, 3.0),
    'drop_threshold': (0.1, 0.9)
}
results_list = []

print(f"準備進行 {iterations} 次隨機參數測試...")

# 4. 執行敏感度分析迴圈
for _ in tqdm(range(iterations), desc="執行進度"):
    # 生成隨機參數
    ma_p = random.randint(*param_ranges['ma_period'])
    bb_p = random.randint(*param_ranges['bb_period'])
    # 確保 bb_period > ma_period
    while bb_p <= ma_p:
        bb_p = random.randint(*param_ranges['bb_period'])

    bb_s = random.uniform(*param_ranges['bb_std'])
    drop_t = random.uniform(*param_ranges['drop_threshold'])

    # 執行回測
    df_result = backtest_strategy(
        df_original.copy(), # 確保每次都使用原始資料
        ma_period=ma_p,
        bb_period=bb_p,
        bb_std=bb_s,
        drop_threshold=drop_t
    )

    # 計算策略績效
    df_result['entry'] = (df_result['position'] == 1) & (df_result['position'].shift(1) == 0)
    df_result['exit'] = (df_result['position'] == 0) & (df_result['position'].shift(1) == 1)
    performance = calculate_strategy_one_performance(df_result)

    # 儲存結果
    run_results = {
        'ma_period': ma_p,
        'bb_period': bb_p,
        'bb_std': round(bb_s, 2),
        'drop_threshold': round(drop_t, 2)
    }
    run_results.update(performance)
    results_list.append(run_results)

print("敏感度分析完成。")

# 5. 整理與呈現結果
if results_list:
    results_df = pd.DataFrame(results_list)

    # 為了方便閱讀，重新命名績效指標的欄位名稱
    rename_dict = {
        'final_equity': '最終權益',
        'net_profit': '淨利',
        'mdd': '最大回撤(MDD)',
        'total_profit': '總獲利',
        'total_loss': '總虧損',
        'total_trades': '總交易次數',
        'winning_trades': '獲利次數',
        'losing_trades': '虧損次數',
        'win_rate': '勝率',
        'max_profit_per_trade': '單次最大獲利',
        'max_loss_per_trade': '單次最大虧損',
        'avg_profit_per_trade': '平均獲利',
        'avg_loss_per_trade': '平均虧損',
        'profit_loss_ratio': '賺賠比',
        'max_consecutive_wins': '最長連續獲利',
        'max_consecutive_losses': '最長連續虧損'
    }
    results_df.rename(columns=rename_dict, inplace=True)
    # 排序找出最佳結果
    # best_results = results_df.sort_values(by='淨利或淨損 (已實現)', ascending=False)

    display(results_df)
else:
    print("沒有任何結果可供分析。")

所有模組載入成功。
資料載入成功，共 1261 筆。
準備進行 100 次隨機參數測試...


執行進度:   0%|          | 0/100 [00:00<?, ?it/s]

敏感度分析完成。


,ma_period,bb_period,bb_std,drop_threshold,最終權益 (Mark-to-Market),淨利或淨損 (已實現),最大回撤 (MDD),總獲利 (已實現),總損失 (已實現),總交易次數,賺錢交易次數,虧錢交易次數,勝率,單次交易最大獲利,單次交易最大損失,獲利交易中的平均獲利,損失交易中的平均損失,賺賠比,最長的連續性獲利的次數,最長的連續性損失的次數
0,3,38,2.45,0.22,87.0,87.0,134.0,151.0,-64.0,6,4,2,66.67%,71.0,-47.0,37.750000,-32.000000,1.179688,3,1
1,4,35,1.56,0.70,-111.0,-127.0,793.0,1430.0,-1557.0,39,11,28,28.21%,254.0,-148.0,130.000000,-55.607143,2.337829,3,9
2,10,16,1.66,0.75,1095.0,1095.0,225.0,1774.0,-679.0,44,25,19,56.82%,232.0,-99.0,70.960000,-35.736842,1.985626,7,4
3,14,17,1.21,0.64,167.0,151.0,1091.0,2529.0,-2378.0,70,23,47,32.86%,289.0,-183.0,109.956522,-50.595745,2.173237,3,12
4,6,30,2.50,0.22,124.0,124.0,29.0,124.0,0.0,3,3,0,100.00%,110.0,0.0,41.333333,0.000000,inf,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3,30,1.78,0.27,463.0,463.0,276.0,1018.0,-555.0,24,11,13,45.83%,233.0,-129.0,92.545455,-42.692308,2.167731,3,4
96,4,20,1.98,0.86,629.0,629.0,110.0,861.0,-232.0,24,15,9,62.50%,172.0,-45.0,57.400000,-25.777778,2.226724,5,3
97,5,22,1.14,0.69,653.0,637.0,758.0,2753.0,-2116.0,62,23,39,37.10%,341.0,-183.0,119.695652,-54.256410,2.206111,3,6
98,5,16,2.24,0.23,162.0,162.0,92.0,293.0,-131.0,11,6,5,54.55%,110.0,-45.0,48.833333,-26.200000,1.863868,2,2
